# load caption

In [5]:
EGO_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d/'
CAPTION_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d_action_caption'
MOVE_CAPTION_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d_move_action_caption'

import json
import os
import tqdm

from video_caption_action_scene import AnnotationLoader, BetaAlphaCalculator

EGO_VERSION_ROOT = os.path.join(EGO_ROOT, 'v2')
json_path = os.path.join(EGO_ROOT, 'ego4d.json')
train_path = f'{EGO_VERSION_ROOT}/annotations/refined_narration_stream_train.json'
val_path = f'{EGO_VERSION_ROOT}/annotations/refined_narration_stream_val.json'
origin_path = f'{EGO_VERSION_ROOT}/annotations/all_narrations_redacted.json'
video_root = f'{EGO_VERSION_ROOT}/full_scale_2fps'
alpha = 4.9
device = 'cuda:3'
caption_dir = '/root/videollm-online/tmp5'

annotation_loader = AnnotationLoader(train_path, val_path, origin_path, json_path)
data = annotation_loader.get_data()
origin_narration = annotation_loader.get_origin_narration()

beta_alpha_calculator = BetaAlphaCalculator(data, alpha)
beta_alpha_calculator.compute_beta()
beta_map = beta_alpha_calculator.get_beta_map()
alpha = beta_alpha_calculator.get_alpha()

train_caption = json.load(open(f'{CAPTION_ROOT}/action_caption_train.json'))
val_caption = json.load(open(f'{CAPTION_ROOT}/action_caption_val.json'))
all_caption = {**train_caption, **val_caption}

move_train_caption = json.load(open(f'{MOVE_CAPTION_ROOT}/action_caption_train.json'))
move_val_caption = json.load(open(f'{MOVE_CAPTION_ROOT}/action_caption_val.json'))
move_all_caption = {**move_train_caption, **move_val_caption}
video2scene = json.load(open('/home/zhangyl/videollm-online/data/estp/ego4d/metafile/video2scene.json'))

# object QA

## move action

In [20]:
def merge_caption_wo_action(captions, video_uid, clip_uid, video2scene, origin_narration, w_pre = False):
    caption = captions[video_uid][clip_uid]
    caption_texts = {}
    for action_idx, cap in enumerate(caption):
        
        caption_texts[action_idx] = {
            'caption': cap['caption'],
        }
        
    return caption_texts, caption

def print_json(caption_texts):
    print(json.dumps(caption_texts, indent=4))
    
def caption_merger(captions, video2scene, origin_narration, w_pre = False):
    for video_uid in captions.keys():
        for clip_uid in captions[video_uid].keys():
            caption_texts, caption = merge_caption_wo_action(captions, video_uid, clip_uid, video2scene, origin_narration, w_pre = False)
            yield caption_texts, caption, video_uid, clip_uid

caption_merger1 = caption_merger(move_all_caption, video2scene, origin_narration)

In [21]:
import json
from openai import OpenAI
import os

def get_llm_response_json(system_prompt, user_prompt):
    client = OpenAI(
            api_key="",
            base_url="https://api.deepseek.com",
        )

    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        response_format={
            'type': 'json_object'
        }
    )
    return response.choices[0].message.content

system_prompt = open('/home/zhangyl/videollm-online/data/estp/ego4d/prompt/caption2object_system_prompt.txt').read().format(NUMBER=2)
user_prompt_templete = open('/home/zhangyl/videollm-online/data/estp/ego4d/prompt/caption2qa_prompt.txt').read()
output_dir = '/home/zhangyl/videollm-online/dataset/move_caption2object_v1/'
os.makedirs(output_dir, exist_ok=True)


for i in range(0,1):

    caption_texts, caption, video_uid, clip_uid = next(caption_merger1)

    user_prompt = user_prompt_templete.format(json.dumps(caption_texts,indent=4))
    # answer = (get_llm_response_json(system_prompt, user_prompt))
    
    with open(os.path.join(output_dir, f'{video_uid}_{clip_uid}_caption.txt'), 'w') as f:
        f.write(user_prompt)
    # with open(os.path.join(output_dir, f'{video_uid}_{clip_uid}_object.txt'), 'w') as f:
    #     f.write(answer)
    
    # answer_json = json.loads(answer)
    # for cap, obj in zip(caption, answer_json.values()):
    #     cap['obj_list'] = obj['object_list']
    
    
    # user_prompt = user_prompt_templete_a.format(caption_texts, answer)
    # answer = get_llm_response_json(system_prompt_a, user_prompt)
    
    # with open(os.path.join(output_dir, f'{video_uid}_{clip_uid}_caption2.txt'), 'w') as f:
    #     f.write(user_prompt)
    # with open(os.path.join(output_dir, f'{video_uid}_{clip_uid}_a.txt'), 'w') as f:
    #     f.write(answer)
    
print_json(caption)
        

[
    {
        "caption": "The video showcases a first-person perspective of an individual engaged in a mechanical or automotive task within a workshop environment. The primary focus is on the person's hands, which are clad in black gloves with red accents and occasionally hold various tools and parts.\n\nInitially, the individual is seen holding a large cylindrical object, likely a part of a vehicle such as a headlight or similar component, against a backdrop of a cluttered workbench covered with a white cloth stained with grease or paint. Tools and materials are scattered around, indicating an active workspace.\n\nAs the video progresses, the individual manipulates the cylindrical object, cleaning it with a yellow cloth. The movement is methodical, suggesting careful handling to ensure thorough cleaning without damaging the surface. The camera remains steady throughout these actions, providing a consistent view of the process.\n\nSubsequently, the scene shifts slightly to show the p

In [19]:
print(video_uid, clip_uid)

1cc9a153-cec3-4ec5-8e85-78c775653bd2 a003760a-fb37-4cbc-9ca8-6db79d3b0029


# function QA

## move action

In [4]:
def merge_caption_wo_action(captions, video_uid, clip_uid, video2scene, origin_narration, w_pre = False):
    caption = captions[video_uid][clip_uid]
    
    caption_texts = "Video Subject : {}\n".format(' / '.join(video2scene[video_uid]))
    for action_idx, cap in enumerate(caption):
        
        if 'stamp_time' not in cap and w_pre:
            action_narration = 'Time is {} - {}.\n'.format(cap['start_time'], cap['end_time'])
        elif 'stamp_time' not in cap and not w_pre:
            continue
        else:
            action_narration = 'Time is {}.\n'.format(cap['stamp_time'])
        caption_text = 'Detailed Description: \"' + cap['text'] + '\".\n'
        caption_text = action_narration + caption_text
        caption_texts += caption_text + '\n\n'
        
    return caption_texts

def caption_merger(captions, video2scene, origin_narration):
    for video_uid in captions.keys():
        for clip_uid in captions[video_uid].keys():
            caption_texts = merge_caption_wo_action(captions, video_uid, clip_uid, video2scene, origin_narration)
            yield caption_texts, video_uid, clip_uid

caption_merger1 = caption_merger(move_all_caption, video2scene, origin_narration)

In [97]:
import json
from openai import OpenAI
import os

def get_llm_response_json(system_prompt, user_prompt):

    client = OpenAI(
        api_key="",
        base_url="https://api.deepseek.com",
    )
    
    messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
    )
    
    return response.choices[0].message.content

system_prompt = open('/home/zhangyl/videollm-online/data/estp/ego4d/prompt/caption2qa_function_system.txt').read().format(NUMBER=2)
user_prompt_templete = open('/home/zhangyl/videollm-online/data/estp/ego4d/prompt/caption2qa_function_prompt.txt').read()
output_dir = '/home/zhangyl/videollm-online/dataset/move_action_function_v12/'
os.makedirs(output_dir, exist_ok=True)

for _ in range(0,10):
    
    caption_texts, video_uid, clip_uid = next(caption_merger1)
    user_prompt = user_prompt_templete.format(caption_texts)
    answer = get_llm_response_json(system_prompt, user_prompt)

    with open(os.path.join(output_dir, f'{video_uid}_{clip_uid}_caption.txt'), 'w') as f:
        f.write(user_prompt)
    with open(os.path.join(output_dir, f'{video_uid}_{clip_uid}_qa.txt'), 'w') as f:
        f.write(answer)    

## action

In [72]:
def merge_caption_with_action(captions, narrations, video_uid, clip_uid, video2scene, origin_narration):
    narration = narrations[video_uid][clip_uid]
    caption = captions[video_uid][clip_uid]
    
    caption_texts = "Video Subject : {}\n".format(' / '.join(video2scene[video_uid]))
    for action_idx, (nar, cap) in enumerate(zip(narration, caption)):
        action_narration = 'Time is {}. Action narration is \"'.format(nar['time']) + nar['text'] + '\".\n'
        caption_text = 'Detailed Description: \"' + cap['text'] + '\" \n'
        caption_text = action_narration + caption_text
        caption_texts += caption_text + '\n'
        
    return caption_texts

def caption_merger_waction(captions, video2scene, origin_narration):
    for video_uid in captions.keys():
        for clip_uid in captions[video_uid].keys():
            caption_texts = merge_caption_with_action(captions, data, video_uid, clip_uid, video2scene, origin_narration)
            yield caption_texts, video_uid, clip_uid

caption_merger2 = caption_merger_waction(all_caption, video2scene, origin_narration)

In [73]:
print(next(caption_merger2)[0])

Video Subject : BBQ'ing/picnics / Walking on street
Time is 270.0. Action narration is "You fold the luggage.".
Detailed Description: "With a sense of purpose, you begin to fold the luggage. The beige fabric of the suitcase is well-worn but still sturdy as it sits on the floor in front of you. You methodically work through the steps: first, you gather the edges and bring them together, then you tuck in the straps, ensuring they are neatly tucked away. As you continue, the once sprawling mass of fabric gradually transforms into a compact, manageable size, ready for transport or storage." 

Time is 301.29231. Action narration is "You look around.".
Detailed Description: "As you look around, the room appears dimly lit with a warm, yellowish hue. The walls are painted in a light beige color, and there's a noticeable shadow cast by an unseen object or person. In front of you, on the floor, lies a large, brown blanket that seems to be spread out, possibly covering something underneath. To yo

# format

In [83]:

def clipuid2cliptime(origin_narrations, video_uid, clip_uid):
    summs = origin_narrations[video_uid]['summaries']
    is_match = False
    for summ in summs:
        if summ['_annotation_uid'] == clip_uid:
            is_match = True
            break
    if not is_match:
        return None, None
    clip_start_time = summ['start_time']
    clip_end_time = summ['end_time']
    return clip_start_time, clip_end_time


def parse_qa_file(file_path, video_uid, clip_uid):
    """
    解析 .txt 文件，将每一组 QA 和任务类型转化为字典
    
    参数:
    - file_path: .txt 文件路径
    
    返回:
    - qa_list: 包含所有 QA 和任务类型的字典列表
    """
    qa_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    step_idx = 0
    for i, line in enumerate(lines):
        if 'step' in line.lower():
            step_idx = i
    lines = lines[step_idx:]
    
    clip_start_time, clip_end_time = clipuid2cliptime(origin_narration, video_uid, clip_uid)
    if clip_end_time is None:
        return []
    current_qa = {}
    current_a = []
    
    for line in lines:
        line = line.strip()
        line = line.replace('*', '')
        if "Task Type:" in line:
            current_qa["Task Type"] = line.split(": ")[1]
        elif "Question:" in line or "Q:" in line:
            current_qa["question"] = line.split(": ")[1].strip()
        elif "Answer:" in line or "A:" in line:
            current_a.append(line.split(":")[1].strip())
        elif "Time:" in line:
            current_a.append(line.split(":")[1].strip())
        elif line == "":
            if current_qa:
                conversation = []
                for i in range(len(current_a)):
                    if i % 2 == 0:
                        beta = beta_map.get(clip_uid, 0)
                        t = float(current_a[i+1])
                        start_time = t - beta / (2 * alpha)
                        end_time = t + beta / (2 * alpha)
                        conversation.append(
                            {
                                'role': 'assistant',
                                'content': current_a[i],
                                'time': t,
                                'start_time': start_time,
                                'end_time': end_time,
                            }
                        )
                current_qa["conversation"] = conversation
                qa_list.append(current_qa)
                current_qa = {}
                current_a = []
    
    # 添加最后一组 QA（如果文件末尾没有空行）
    if current_qa:
        conversation = []
        for i in range(len(current_a)):
            if i % 2 == 0:
                beta = beta_map.get(clip_uid, 0)
                try:
                    t = float(current_a[i+1])
                except:
                    print(video_uid, clip_uid, current_a)
                start_time = t - beta / (2 * alpha)
                end_time = t + beta / (2 * alpha)
                conversation.append(
                    {
                        'role': 'assistant',
                        'content': current_a[i],
                        'time': t,
                        'start_time': start_time,
                        'end_time': end_time,
                    }
                )
        current_qa["conversation"] = conversation
        qa_list.append(current_qa)
        
        
    for qa in qa_list:
        qa["clip_start_time"]=clip_start_time
        qa["clip_end_time"]=clip_end_time
    return qa_list

output_dir = '/home/zhangyl/videollm-online/dataset/move_action_function_v8/'
postfix = 'qa.txt'
file_list = os.listdir(output_dir)
annos = {}
for file in file_list:
    if file.endswith(postfix):
        video_uid, clip_uid = file.split('_')[:2]
        if video_uid not in annos:
            annos[video_uid] = {}
        annos[video_uid][clip_uid] = parse_qa_file(os.path.join(output_dir, file),
                                                   video_uid, clip_uid)

with open('/home/zhangyl/videollm-online/data/estp/annotations/move_action_function_v8.json', 'w') as f:
    json.dump(annos,f, indent=4)
